In [1]:
#!/usr/bin/env python3
# coding=utf-8
import pandas as pd
import numpy as np
import sqlite3
import time
from datetime import datetime, timedelta
import math
import sys
import logging
import os
# from scipy import stats as st
from scipy import stats 
from itertools import product as product
from matplotlib import pyplot as plt
# from sklearn.linear_model import LogisticRegression
import warnings
import statsmodels.api as sm
from math import sqrt
from tqdm import tqdm
import pymssql

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

# conn = pymssql.connect("127.0.0.1", "sa", "quant@123", "test_block")
# cursor = conn.cursor()

g_worthDir = '../pkl'  # 
g_parentDir = '../../..'  # 
g_dbfile = f"{g_parentDir}/pkl/hq.db" # 
g_name = g_access = 'sr_min'

g_locTimeadj = 28800 # 28800 0
g_version = '1.0.1'

print(f"{sys.version} , {sys.executable} , pandas: {pd.__version__}")
print(f"{time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()+g_locTimeadj))} *****import**v{g_version}*** ")

In [2]:
###日志 logger##20220825A######
def make_logger(name, log_level=logging.INFO, log_file="log.txt", file_mode="a"): #w写 a追加
    formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s:%(process)s: %(message)s')
    
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(level=log_level)
        handler = logging.FileHandler(log_file, mode=file_mode)
        handler.setLevel(level=log_level)
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        handler = logging.StreamHandler(sys.stdout)
        handler.setLevel(level=log_level)
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger

logger = make_logger(g_name, log_level=logging.DEBUG, log_file= f"{g_parentDir}/logs/{g_name if ('g_name' in dir() ) else 'test' }_{ time.strftime('%Y%m%d') }.log" )

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****logging***** ')

2022-12-08 19:30:58 *****logging***** 


In [ ]:
# ukdf.to_pickle(f"{g_worthDir}/uk5mdf.pkl" )
# ##df_uk = pd.read_pickle(f"{g_worthDir}/uk5mdf.pkl" )

df_uk = pd.read_pickle(f"{g_parentDir}/pkl/uk5mdf.pkl" ) # uk5m_2021df 203451
# df_uk.tail(2)
# print(f"{df_uk.iloc[-2:,:]}")
# display(df_uk.tail(2) )
print(f"df_uk.tail(2) :\n{df_uk.tail(2)}")

df_input = pd.DataFrame()
# df_input['date'] = df_uk['closeTime'].apply(lambda x: time.strftime("%Y%m%d",time.localtime(float(x)/1000) ) )
df_input['date'] = df_uk['tradeDate']
df_input['timestamp'] = df_uk['closeSec']
df_input['closeTime'] = df_uk['closeTime']
df_input['open'] = df_uk['open']
df_input['high'] = df_uk['high']
df_input['low'] = df_uk['low']
df_input['close'] = df_uk['close']
df_input['vol'] = df_uk['vol']
df_input['amount'] = df_uk['amt']
df_input['amt'] = df_uk['amt']/( df_uk['high']+ df_uk['low'])*2

# df_input.tail(2)
# display(df_input.tail(2) )
print(f"df_input.tail(2) :\n{df_input.tail(2)}")


In [3]:
if_print = 0
wid_ = np.arange(12)*4*12 + 144  # 12: [144 192 240 288 336 384 432 480 528 576 624 672]
thd = np.arange(31)*0.01 +0.6   # 31: [0.6  0.61 ... 0.88 0.89 0.9 ]
wid_back = np.arange(10)*5 +10  # 10: [10 15 20 25 30 35 40 45 50 55]
thd_std = np.arange(16)*0.2 + 2 # 16: [2. 2.2 2.4 2.6 2.8 3. 3.2 3.4 3.6 3.8 4. 4.2 4.4 4.6 4.8 5.]
df_para = pd.DataFrame(list(product(wid_, thd,wid_back,thd_std)))
df_para.columns = ['wid_','thd','wid_back','thd_std'] # 59520

print(f"wid_ : {len(wid_)}: {wid_}") 
print(f"wid_back : {len(wid_back)}: {wid_back}")
print(f"thd : {len(thd)}: {thd}")
print(f"thd_std : {len(thd_std)}: {thd_std}")
print(f"df_para : {len(df_para)}: {df_para}")

df_para['re']=np.nan
df_para['std']=np.nan
# print(df_para)
for index_para,row_para in tqdm(df_para.iterrows(),total=df_para.shape[0]):

    if index_para!=9450:
        if_print = 1
        continue
    wid_,thd=int(row_para[0]),row_para[1]
    print(f"{index_para=} : {wid_=} : {thd=} : wid_back={int(row_para[2])} : thd_std={row_para[3]}") # index_para=9450 : wid_=192 : thd=0.88 : wid_back=10 : thd_std=4.0

wid_ : 12: [144 192 240 288 336 384 432 480 528 576 624 672]
wid_back : 10: [10 15 20 25 30 35 40 45 50 55]
thd : 31: [0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73
 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87
 0.88 0.89 0.9 ]
thd_std : 16: [2.  2.2 2.4 2.6 2.8 3.  3.2 3.4 3.6 3.8 4.  4.2 4.4 4.6 4.8 5. ]
df_para : 59520:        wid_  thd  wid_back  thd_std
0       144  0.6        10      2.0
1       144  0.6        10      2.2
2       144  0.6        10      2.4
3       144  0.6        10      2.6
4       144  0.6        10      2.8
...     ...  ...       ...      ...
59515   672  0.9        55      4.2
59516   672  0.9        55      4.4
59517   672  0.9        55      4.6
59518   672  0.9        55      4.8
59519   672  0.9        55      5.0

[59520 rows x 4 columns]


 25%|██▌       | 14964/59520 [00:00<00:01, 31944.63it/s]

index_para=9450 : wid_=192 : thd=0.88 : wid_back=10 : thd_std=4.0


100%|██████████| 59520/59520 [00:02<00:00, 28975.37it/s]
